In [1]:
### Imports for the Data Preprocessing
import numpy as np
import pandas as pd
import matplotlib as plt
import os
import gc
from typing import Tuple
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

import numpy as np

In [2]:
#working directory
# set correct working directory
wd = os.getcwd()
print("Current working directory: ", wd)
if wd[-9:] == "notebooks": wd = wd[:-10]
else: print("Make sure your working directory is set to the student-performance folder!")
os.chdir(wd)

Current working directory:  c:\Users\Stephan\Documents\Python\Python-Stat\Neuer\Kaggle_semiinar\student-performance\notebooks


In [3]:
from utils.loader_steve import load_data

data = load_data("data\processed\df_level.csv", n_rows= 10000)

In [23]:
import pandas as pd

def mean_level_values(df):
    # Group the DataFrame by 'level' column and calculate the mean for each group
    mean_df = df.groupby('level').mean().reset_index()
    
    return mean_df


In [8]:
data_mean_level = pd.read_csv("data/processed/df_mean_level.csv")
#df_mean_level = mean_level_values(data)
#df_mean_level.to_csv('data/processed/df_mean_level.csv')

In [48]:
import pandas as pd
import numpy as np

def generate_rows(df: pd.DataFrame, n_flatten: int, level_g: str, mean_value_levels: pd.DataFrame):
    counts = df['session_id'].value_counts()
    df['level'] = df['level'].astype(np.uint8)

    if (counts % n_flatten).any():
        need_generating = counts[counts < n_flatten].index.tolist()
        num_generated_rows = 0
        generated_sessions = []
        generated_rows = []

        for session_id in need_generating:
            levels_present = set(df.loc[df['session_id'] == session_id, 'level'].unique())

            level_group_first = df.loc[df['session_id'] == session_id, 'level_group'].str.split('-').str[0].values[0]
            level_group_last = df.loc[df['session_id'] == session_id, 'level_group'].str.split('-').str[-1].values[0]
            level_group_first = int(level_group_first)
            level_group_last = int(level_group_last)
            expected_num_rows = level_group_last - level_group_first + 1

            if len(df.loc[df['session_id'] == session_id]) != expected_num_rows:
                expected_levels = set(range(level_group_first, level_group_last + 1))
                missing_levels = expected_levels - levels_present

                new_rows = []
                for missing_level in missing_levels:
                    new_row = {
                        "session_id": session_id,
                        "level": missing_level,
                        "level_group": str(level_group_first)
                    }

                    for col in df.columns:
                        if col in ("session_id", "level", "level_group"):
                            continue

                        else:
                            mean_value = mean_value_levels.loc[missing_level, col]
                            new_row[col] = mean_value

                    new_rows.append(new_row)

                df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
                num_generated_rows += len(new_rows)
                generated_sessions.append({"session_id": session_id, "num_rows": len(new_rows)})
                generated_rows.extend(new_rows)

        print(f"Generated {num_generated_rows} rows with indices: {list(range(len(df) - num_generated_rows, len(df)))}")

        df2 = pd.DataFrame(generated_sessions)
        df2 = df2.set_index("session_id")
        print("Generated rows per session id:")
        print(df2)

        df3 = pd.DataFrame(generated_rows)
        df3 = df3.reindex(df.columns, axis=1)
        print("Generated rows:")
        print(df3)
    else:
        df2 = pd.DataFrame()
        df3 = pd.DataFrame()

    df["level_group"] = level_g
    df = df.sort_values(by=["session_id", "level"])
    df = df.reset_index(drop=True)
    df3["level_group"] = level_g

    return df, df2, df3


In [5]:
import pandas as pd

def split_dataframe(df, column):
    # Group the DataFrame by the specified column
    groups = df.groupby(column)
    
    # Create empty DataFrames to store the results
    dfs = []
    
    # Iterate over the groups and store each group as a separate DataFrame
    for name, group_df in groups:
        dfs.append(group_df)
    
    return tuple(dfs)


In [58]:
# Assuming your original DataFrame is called 'df'
df_1, df_2, df_3 = split_dataframe(data, 'level_group')
df_2.reset_index(inplace = True)
df_1.reset_index(inplace = True)
df_3.reset_index(inplace = True)




In [18]:
print(df_2.index.name)

session_id


In [70]:

df, df2, df3 = generate_rows(df_3.iloc[[4]],8,"5-12",data_mean_level)

Generated 7 rows with indices: [1, 2, 3, 4, 5, 6, 7]
Generated rows per session id:
                   num_rows
session_id                 
20090312431273200         7
Generated rows:
          session_id  level level_group  event_name      name       fqid  \
0  20090312431273200      5           5    4.296876  2.300357   6.672311   
1  20090312431273200      6           5    4.758552  3.072108  10.719464   
2  20090312431273200      7           5    7.163236  3.332117  11.249523   
3  20090312431273200      8           5    6.588235  3.285629   5.458450   
4  20090312431273200     10           5    4.559630  3.345896   3.187420   
5  20090312431273200     11           5    7.374544  3.419404  14.512308   
6  20090312431273200     12           5    4.297301  2.640905   5.719082   

   room_fqid  text_fqid  fullscreen        hq  ...  page_std  \
0   4.344580   1.379000    0.138783  0.120278  ... -0.927310   
1   5.140820   8.657330    0.138783  0.120278  ... -0.506965   
2   4.189041   

C:\Users\Stephan\AppData\Local\Temp\ipykernel_2384\223129343.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['level'] = df['level'].astype(np.uint8)


In [62]:
check = df['session_id'].value_counts()
print(max(check))
num_l = levels = df["level"].value_counts()
#print(num_l)

8
